In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from sklearn.linear_model import LinearRegression
import pickle
import unidecode
from bs4 import BeautifulSoup
import requests
import time
import matplotlib.pyplot as plt

In [72]:
# Team Map for predicted vorps DF
team_map = {
    'ATL' : 'Atlanta Hawks',
    'BOS' : 'Boston Celtics',
    'BRK' : 'Brooklyn Nets',
    'CHI' : 'Chicago Bulls',
    'CHO' : 'Charlotte Hornets',
    'CLE' : 'Cleveland Cavaliers',
    'DAL' : 'Dallas Mavericks',
    'DEN' : 'Denver Nuggets',
    'DET' : 'Detroit Pistons',
    'GSW' : 'Golden State Warriors',
    'HOU' : 'Houston Rockets',
    'IND' : 'Indiana Pacers',
    'LAC' : 'Los Angeles Clippers',
    'LAL' : 'Los Angeles Lakers',
    'MEM' : 'Memphis Grizzlies',
    'MIA' : 'Miami Heat',
    'MIL' : 'Milwaukee Bucks',
    'MIN' : 'Minnesota Timberwolves',
    'NOP' : 'New Orleans Pelicans',
    'NYK' : 'New York Knicks',
    'OKC' : 'Oklahoma City Thunder',
    'ORL' : 'Orlando Magic',
    'PHI' : 'Philadelphia 76ers',
    'PHO' : 'Phoenix Suns',
    'POR' : 'Portland Trail Blazers',
    'SAC' : 'Sacramento Kings',
    'SAS' : 'San Antonio Spurs',
    'TOR' : 'Toronto Raptors',
    'UTA' : 'Utah Jazz',
    'WAS' : 'Washington Wizards'
}

# Function to change Sixers name for predicted_vorps table
def sixers_change(x):
    if x == 'Philadelphia Sixers':
        return "Philadelphia 76ers"
    else:
        return x
    
# Importing necessary prediction models
file_name = 'Model_Build/Data/vorp_regression.pickle'
with open(file_name, 'rb') as f:
    vorp_pt_model = pickle.load(f)
    
file_name = 'Model_Build/Data/win_pct_regression.pickle'
with open(file_name, 'rb') as f:
    point_win_pct_model = pickle.load(f)

In [36]:
# Import necessary data and aligning team names
prior_year_vorps = pd.read_csv('In_Season/Data/prior_year_vorps.csv', index_col = 0)
prior_year_vorps = prior_year_vorps[prior_year_vorps.Team != 'TOT']
prior_year_vorps['Team'] = prior_year_vorps.Team.apply(lambda x: team_map[x])

predicted_vorps = pd.read_csv('In_Season/Data/opening_day_vorps.csv', index_col = 0)
predicted_vorps['Team'] = predicted_vorps.Team.apply(sixers_change)

prior_year_differential = pd.read_csv('In_Season/Data/prior_year_adjusted_win_pct.csv', index_col = 0)

In [53]:
# Doing merges

merged_1 = pd.merge(prior_year_vorps, predicted_vorps, on = 'Team')
merged_1.columns = ['Team', 'Prior_Year_VORP', 'VORP_Projection']

df = pd.merge(merged_1, prior_year_differential, on = 'Team')
df = df[['Team', 'Prior_Year_VORP', 'VORP_Projection', 'Adj_Point_Differential_82']]
df.columns = ['Team', 'Prior_Year_VORP', 'VORP_Projection', 'Prior_Year_Differential']

In [61]:
df['VORP_Projection'] = df.VORP_Projection.apply(lambda x: float(x.strip("'[]")) if type(x) == str else x)

In [73]:
# Calculating point adjustment based on VORP differential
df['VORP_Difference'] = df.VORP_Projection - df.Prior_Year_VORP
coefficient = vorp_pt_model.coef_[0]
df['Point_Adjustment'] = coefficient * df.VORP_Difference

# Calculating projected win % based on projected point differential
df['Projected_Point_Differential'] = df.Prior_Year_Differential + df.Point_Adjustment
x = df[['Projected_Point_Differential']]
df['Projected_Win_Pct'] = point_win_pct_model.predict(x)
df

,Team,Prior_Year_VORP,VORP_Projection,Prior_Year_Differential,VORP_Difference,Point_Adjustment,Projected_Point_Differential,Projected_Win_Pct
0,Atlanta Hawks,12.413889,16.368954,195.74548,3.955065,216.053403,411.798883,0.656887
1,Boston Celtics,11.502778,13.891392,113.68808,2.388614,130.482860,244.170940,0.592687
2,Brooklyn Nets,15.147222,17.485986,381.75920,2.338764,127.759673,509.518873,0.694313
3,Chicago Bulls,8.769444,16.885983,-75.59006,8.116538,443.382260,367.792200,0.640033
4,Charlotte Hornets,7.630556,9.243450,-177.54804,1.612895,88.107611,-89.440429,0.464917
5,Cleveland Cavaliers,0.113889,8.781508,-719.43520,8.667620,473.486178,-245.949022,0.404975
6,Dallas Mavericks,12.755556,13.192963,192.39660,0.437408,23.894275,216.290875,0.582009
7,Denver Nuggets,16.058333,16.172952,411.63508,0.114618,6.261266,417.896346,0.659222
8,Detroit Pistons,4.669444,7.017302,-389.53116,2.347858,128.256470,-261.274690,0.399106
9,Golden State Warriors,11.275000,11.943167,91.52512,0.668167,36.499963,128.025083,0.548204
